# Chiral solutions for Type-II Dirac seesaw with a Active Symmetry
![img](https://raw.githubusercontent.com/restrepo/anomaly/main/schema.svg)

In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
pd.set_option('display.max_colwidth',500)

### Load full solutions

In [3]:
ds=pd.read_json('https://github.com/restrepo/anomaly/raw/main/solutions.json.gz')
ds.shape

(390074, 5)

## Filter two set of repeated charges
with one of them repeated 3 times
$$(\nu,\nu[,\nu],m,m,m,\psi_1,\psi_2,\cdots)$$


In [4]:
#at least two set of repeated numbers
dsp=ds[ds['solution'].apply(lambda l: len(l)-len(set(l))>2)].reset_index(drop=True)
dsp.shape

(48290, 5)

In [5]:
dsp=dsp[dsp['solution'].apply(lambda l: 3 in [ y for y in [l.count(x) for x in set(l) if l.count(x)!=1] 
                                  if y==2 or y==3
                                ])].reset_index(drop=True)

In [6]:
dsp['solution']

0           [1, 4, 4, 4, -5, -5, -5, -5, -5, 6, 6]
1                         [1, 2, 2, -3, -3, -3, 4]
2         [1, 1, 1, 1, 2, 2, 2, -3, -4, -4, -6, 7]
3           [1, 1, 1, -2, -2, -5, -5, 6, 6, 7, -8]
4        [1, 1, 1, -2, 6, 6, 6, -7, -7, -7, -7, 9]
                           ...                    
21462          [3, 3, 3, 4, -14, 26, -27, -27, 29]
21463                [9, 9, -24, -24, -24, 25, 29]
21464         [13, -17, -17, -17, 20, 20, 23, -25]
21465         [13, 13, -15, -15, -15, 20, 27, -28]
21466             [5, 5, 5, -17, -27, -27, 28, 28]
Name: solution, Length: 21467, dtype: object

Some reordering

In [7]:
dsp['nmax']=dsp['solution'].apply(lambda l: map(abs,l)).apply(max)
dsp=dsp.sort_values(['n','nmax']).reset_index(drop=True)
#Show the first and last one solutions
dsp.iloc[[0,1,2,-1]]

,l,k,solution,gcd,n,nmax
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,5
1,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,7,4
2,"[-3, -1]","[-2, -3, 1]","[3, 3, 3, -5, -5, -7, 8]",1,7,8
21466,"[-2, -3, -2, -3, -2]","[-1, 3, -1, 3, -1]","[14, -18, -24, -24, -24, 27, 27, 28, 29, 29, -32, -32]",2,12,32


## 1. Check phenomenological conditions
<div style="float: right;" markdown="1">
    <img src="https://raw.githubusercontent.com/restrepo/anomaly/main/TII-seesaw.svg">
</div>

Notation:
* $f_R \to f$
* $(f_L)^\dagger \to f$
* $S\to s$
* $H\to h$
* $m=L-h$, where $L$ is the charge of the anti-Lepton doublet.

$$(\nu,\nu[,\nu],m,m,m,\psi_1,\psi_2,\cdots)$$


$$ \nu+m+s=0$$

$$ \psi_i+\psi_j=|s|$$

In [8]:
def get_massles(l,s):
    if len(l)==0:
        return []
    elif len(l)==1:
        mssls=[(l[0],l[0])]
    elif len(l)==2:
        mssls=[tuple(l)]
    else:
        mssls=set([x for x in itertools.permutations(l,2) if x[0]<=x[1] ])
    xs=[ x for x in mssls if abs(x[0]+x[1])==abs(s) ]
    #flatten list of list into list
    massive=[x for sublist in xs for x in sublist]

    return list( set(l).difference(set(massive)) )    

def get_sp(massless):
    """
    For a list of numbers `x_i` find the best `sp` value such that
    all the elements of the list satisfy:
       |sp|=|x_i + x_j|
    * If the list is already empty return None
    * If not found `sp` returns 0
    """
    if len(massless)==0:
        return None
    elif len(massless)==1:
        drs=[(massless[0],massless[0])]
    elif len(massless)==2:
        drs=[tuple(massless)]
    else:
        drs=[x for x in itertools.permutations(massless,2) if x[0]<=x[1]]
    sps=set(abs(sum(x)) for x in drs)    
    for sp in sps:
        msv =([dr for dr in drs if abs(sum(dr))==sp])
        massive=set([x for sublist in msv for x in sublist])
        msl=([dr for dr in drs if abs(sum(dr))!=sp])
        massless=set( [x for sublist in msl for x in sublist])
        majorana=massless.difference(massive)
        majorana=[m for m in majorana if abs(2*m)!=sp]
        if not majorana:
            return sp
        else:
            return 0        

In [9]:
Type='TII-X'
#l=[1,-9,-9,-9,13,18,18,18,-19,-22]
#def conditions(l):
def condition(l,Type='TII-X'):
#if True:   
    sls=[]
    ms=set([x for x in l if l.count(x)==3])
    νs=set([x for x in l if l.count(x)==2 or l.count(x)==3])
    mνs=[(m,ν) for m in ms for ν in νs if m!=ν]
    sold=0
    for mν in mνs:
        m=mν[0]
        ν=mν[1]
        s=-m-ν
        if abs(s)==abs(sold):
            continue
        else:
            sold=s            
        ψs=[x for x in l if x!=ν and x!=m]
        msslss=get_massles(ψs,s)
        sp=get_sp(msslss)
        d={'Type':Type,'m':m,'ν':ν,'s':s,'massless':msslss,'sp':sp}
        sls.append(d)
    #sp=None    
    solutions=[d for d in sls if d.get('massless')==[]]
    if not solutions:
        solutions=[d for d in sls if d.get('sp')]    
    return solutions

l=[1, 2, -6, -6, -6, 8, 9, 9, -11]
assert condition(l)[0].get('massless')==[]
l=[1, 1, 1, -4, -4, 5]
assert condition(l)[0].get('sp')==10
l=[1,-9,-9,-9,13,18,18,18,-19,-22]
assert condition(l)[0].get('sp')==18

In [10]:
dsp['tree-level']=dsp['solution'].apply(condition)

In [11]:
dsp=dsp[dsp['tree-level'].apply(len)>0].reset_index(drop=True)

In [12]:
dsp[dsp['tree-level'].apply(lambda l: [d for d in l if not d.get('massless')]).apply(len)>0]

,l,k,solution,gcd,n,nmax,tree-level
4,"[1, -3, -2]","[-4, -9, -5, -3]","[1, 3, 3, 3, -5, -7, -7, 9]",1,8,9,"[{'Type': 'TII-X', 'm': 3, 'ν': -7, 's': 4, 'massless': [], 'sp': None}]"
29,"[3, 0, -1, 1]","[-6, -5, 5, -1]","[2, 2, 2, -3, -3, 4, -5, -5, 6]",1,9,6,"[{'Type': 'TII-X', 'm': 2, 'ν': -3, 's': 1, 'massless': [], 'sp': None}]"
37,"[1, 2, 1, -1]","[-6, -3, -5, -3]","[1, 2, -6, -6, -6, 8, 9, 9, -11]",1,9,11,"[{'Type': 'TII-X', 'm': -6, 'ν': 9, 's': -3, 'massless': [], 'sp': None}]"
46,"[-2, -1, 0]","[-1, -3, -5, -3]","[5, 6, 6, 6, -8, -9, -9, -10, 13]",2,9,13,"[{'Type': 'TII-X', 'm': 6, 'ν': -9, 's': 3, 'massless': [], 'sp': None}]"
51,"[-2, -1, 0]","[-1, -4, -5, -4]","[1, -5, 8, 8, 8, -11, -12, -12, 15]",2,9,15,"[{'Type': 'TII-X', 'm': 8, 'ν': -12, 's': 4, 'massless': [], 'sp': None}]"
...,...,...,...,...,...,...,...
4674,"[1, -3, 1, 2, 1]","[-3, -1, -3, -1, -3]","[5, 9, -13, -13, -13, -15, -19, 21, 23, 23, 23, -31]",4,12,31,"[{'Type': 'TII-X', 'm': -13, 'ν': 23, 's': -10, 'massless': [], 'sp': None}]"
4701,"[-1, -2, -1, -2, -1]","[-2, 0, -2, 2, -2]","[1, -2, -15, 16, -28, -28, 29, 29, 29, -30, 31, -32]",1,12,32,"[{'Type': 'TII-X', 'm': 29, 'ν': -28, 's': -1, 'massless': [], 'sp': None}]"
4705,"[-2, 2, -2, 1, -2]","[-1, -2, -1, 2, -1]","[2, -4, -4, -4, -7, 11, -14, -23, 25, 25, 25, -32]",3,12,32,"[{'Type': 'TII-X', 'm': 25, 'ν': -4, 's': -21, 'massless': [], 'sp': None}]"
4706,"[-1, -2, -1, -2, 1]","[-1, -2, 2, -2, 2]","[13, 13, -16, -16, -20, -20, 23, 29, 29, 29, -32, -32]",2,12,32,"[{'Type': 'TII-X', 'm': 29, 'ν': -32, 's': 3, 'massless': [], 'sp': None}]"


In [260]:
#zn=dsp[dsp['tree-level'].apply(lambda l: any( [True for d in l if abs(d.get('s'))==6 or abs(d.get('s'))==10 ]))]

In [14]:
def unconditional_stability(row,ZN={6:(2,3),10:(2,5),15:(3,5)},DMs={},check_massless=True):
    '''
    According to appendix of arXiv:1911.05515, 
    generalized to any N
    
    '''
    if check_massless:
        sd=[d for d in row['tree-level'] if d.get('massless')==[]]
    else:
        sd=row['tree-level']
        
    for i in range(len(sd)):
        DMs={}
        d=sd[i]
        N=abs(d['s'])
        l=[n for n in row['solution'] if n not in [d['m'],d['ν']] ]
        #Check for non-hidden chiral fermions
        if 0 in np.array(l)%N:
            DMs={'None':tuple([])}
            continue
        DM=[x for x in itertools.permutations(l,2) 
               if x[0]<=x[1] and abs(sum(x))==N ]
        DM=DM+[(n,n) for n in l if abs(2*n)==N]
        #rank 0
        for dm in DM:
            if dm[0]!=dm[1] and l.count(dm[0])!=l.count(dm[1]) and abs(2*dm[0])!=N and abs(2*dm[0])!=N:
                DMs={'None':tuple([])}
        #missing massles → TODO: Check BUG
        if check_massless and np.setdiff1d(l,[item for sublist in DM for item in sublist]).size>0:
            DMs={'None':tuple([])}
        if not DMs.get('None')==():
            try:
                p,q=ZN[N]
            except KeyError:
                DMs={'None':tuple([])}
                continue
            ps=[n*p for n in range(1,N) if n*p<N]
            qs=[n*q for n in range(1,N) if n*q<N]
            ω=[ sorted([y%N for y in x]) for x in DM]
            DMF=['ψ' if np.setdiff1d(x,ps).size==0 else 'χ' 
                   if np.setdiff1d(x,qs).size==0 else 'None' for x in  ω ]
            DMs=dict(zip(DMF,DM))
            break
    return DMs

In [15]:
p=np.array( [2,3,5,7,11,13,17,19,23,29,31] )
q=p.copy()
zpq= [ pp*qq for pp in p for qq in q if pp<qq] 
Zpq=sorted( set( zpq ) )

In [16]:
pqs=[ (pp,qq) for pp in p for qq in q if pp<qq]
ZN=dict(zip(Zpq,pqs))

In [17]:
Zpq[:3]

[6, 10, 14]

In [18]:
dsp['DMs']=dsp.apply(lambda row: unconditional_stability(row,ZN)  ,axis='columns')

In [19]:
dsp[dsp['DMs'].apply(lambda d: 'ψ' in d.keys() and 'χ' in d.keys())]

,l,k,solution,gcd,n,nmax,tree-level,DMs
1059,"[-2, 0, 1, 0, -3]","[-2, -1, 0, -1, 1]","[1, 3, 5, -6, -6, -6, 8, -9, 12, 12, -14]",2,11,14,"[{'Type': 'TII-X', 'm': -6, 'ν': 12, 's': -6, 'massless': [], 'sp': None}]","{'None': (1, 5), 'χ': (3, 3), 'ψ': (-14, 8)}"
1120,"[-2, 0, -1, 0, 3]","[-2, -1, 0, -1, 1]","[1, -2, 6, 6, 6, -7, 8, -9, -12, -12, 15]",2,11,15,"[{'Type': 'TII-X', 'm': 6, 'ν': -12, 's': 6, 'massless': [], 'sp': None}]","{'ψ': (-2, 8), 'None': (-7, 1), 'χ': (-9, 15)}"
1224,"[-3, 0, -2, 0]","[-2, -4, -2, 1, -1]","[1, 3, 3, 6, 6, 6, -7, -10, -12, -12, 16]",6,11,16,"[{'Type': 'TII-X', 'm': 6, 'ν': -12, 's': 6, 'massless': [], 'sp': None}]","{'χ': (3, 3), 'None': (-7, 1), 'ψ': (-10, 16)}"
1563,"[2, -1, 2, 0]","[-3, -2, -3, -2, -3]","[2, -6, -6, -6, -8, -11, 12, 12, 15, 17, -21]",6,11,21,"[{'Type': 'TII-X', 'm': -6, 'ν': 12, 's': -6, 'massless': [], 'sp': None}]","{'ψ': (-8, 2), 'None': (-11, 17), 'χ': (-21, 15)}"
3730,"[-3, 0, -3, -1, 3]","[-3, -1, -3, -1, 1]","[2, 6, 6, 6, -7, -16, -20, -20, -20, 21, 21, 21]",8,12,21,"[{'Type': 'TII-X', 'm': -20, 'ν': 6, 's': 14, 'massless': [], 'sp': None}, {'Type': 'TII-X', 'm': 6, 'ν': -20, 's': 14, 'massless': [], 'sp': None}]","{'χ': (-7, -7), 'ψ': (-16, 2)}"
